*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [36]:
# import libraries
#try:
#  # %tensorflow_version only exists in Colab.
#  !pip install tf-nightly
#except Exception:
#  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
#!pip install tensorflow-datasets
#import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.8.0


In [37]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2022-03-11 21:01:03--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.2’

train-data.tsv.2    100%[===================>] 349.84K  --.-KB/s    in 0.07s   

2022-03-11 21:01:03 (5.21 MB/s) - ‘train-data.tsv.2’ saved [358233/358233]

--2022-03-11 21:01:03--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.2’

valid-data.tsv.2    100%[========

In [38]:
layers = keras.layers

# Data: loading and preprocessing

In [39]:
BUFFER_SIZE = 1000
BATCH_SIZE = 64
VOCAB_SIZE = 8000
EMBEDDING_DIM = 8
RNN_UNITS = 8
N_EPOCHS = 5

labels = {'ham':0, 'spam':1}

df_train = pd.read_csv(train_file_path, sep='\t', names=['label','message'])
df_test = pd.read_csv(test_file_path, sep='\t', names=['label','message'])

print(f'Train dataset length:\t{len(df_train)}',
      f'Test dataset length:\t{len(df_test)}',
      sep='\n')

Train dataset length:	4179
Test dataset length:	1392


In [40]:
training_data = df_train.replace(labels)
test_data = df_test.replace(labels)

In [41]:
training_data = tf.data.Dataset.from_tensor_slices((training_data.message, training_data.label))
test_data = tf.data.Dataset.from_tensor_slices((test_data.message, test_data.label))

In [42]:
training_data = training_data.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_data = test_data.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Model

In [43]:
encoder = layers.TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(training_data.map(lambda m,l: m))

In [44]:
model = keras.Sequential([
                          encoder,
                          layers.Embedding(input_dim = VOCAB_SIZE,
                                           output_dim = EMBEDDING_DIM,
                                           mask_zero = True),
                          layers.Bidirectional(keras.layers.LSTM(RNN_UNITS,
                                                                 dropout=0.5)),
                          layers.Dense(64, activation='relu'),
                          layers.Dense(1, activation='sigmoid')
])

In [45]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [46]:
model.fit(training_data, epochs=N_EPOCHS)

Epoch 1/5
66/66 [==============================] - 13s 57ms/step - loss: 0.4852 - accuracy: 0.8615
Epoch 2/5
66/66 [==============================] - 3s 46ms/step - loss: 0.2229 - accuracy: 0.9069
Epoch 3/5
66/66 [==============================] - 4s 56ms/step - loss: 0.0910 - accuracy: 0.9816
Epoch 4/5
66/66 [==============================] - 4s 66ms/step - loss: 0.0477 - accuracy: 0.9871
Epoch 5/5
66/66 [==============================] - 3s 51ms/step - loss: 0.0353 - accuracy: 0.9907


In [47]:
model.evaluate(test_data)

22/22 [==============================] - 3s 16ms/step - loss: 0.2384 - accuracy: 0.9188


[0.23840972781181335, 0.9188218116760254]

# Final tests

In [48]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    result = model.predict([pred_text]).item(0)
    label = 'ham' if result<0.5 else 'spam'
    return [result, label]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

[0.0006901894812472165, 'ham']


In [49]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()

You passed the challenge. Great job!
